# Modules

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, date
import os
import csv
import random
import requests
import time

import pandas as pd
import numpy as np
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import time
import random
from fake_useragent import UserAgent

from datetime import datetime, date
import re
from collections import defaultdict

import numpy as np
import openpyxl
from main.main_doc import main_func

In [1]:
Tiker = 'OI'
files = main_func(Tiker)

# Selection Reports

In [12]:
def check_threee_moth_ended(data_frame):
    
    count_ended = {}
    check_columns = []
    have_found =0 
    for indexx,i in enumerate(data_frame.columns):
        second_row = data_frame[i][0]    
        if re.search('Months Ended',i):
            count_ended[indexx] = i
            have_found +=1
    for indexx,i in enumerate(data_frame.columns):
        second_row = data_frame[i][0]    
        if have_found>0:
            data_frame.set_index
        
            try:
                
                new_columns = data_frame.columns[indexx] + ' ' +  second_row
 
            except:
                new_columns = data_frame.columns[indexx]

            check_columns.append(new_columns)

        else:
            
            check_columns.append(i)
            
    data_frame.columns  = check_columns
    if have_found >0:
        data_frame = data_frame.iloc[1:]

    else:
        pass

    
    return data_frame#.iloc[:,1:]

In [13]:
%%time

excl_add = 'Condensed'
exclude_report_list_before_join = ['Net Income per Share',
                       'Summary of Significant Accounting Policies','Common Stock'] 
exclude_report_list = '|'.join(exclude_report_list_before_join)
excludeded_keys = []

data_dict = defaultdict(list)   
REALY_NAMES =  {}
files = main_func(Tiker)
for period in files:
    
    wb = openpyxl.load_workbook(period)
    sheetnames_list = wb.sheetnames

    for sheetname in sheetnames_list:
        sheet = wb[sheetname]

        if len(str(sheet['b4'].value))<9 and len(str(sheet['b3'].value))<9:

            header = sheet['A1'].value
        
 

            if re.search(exclude_report_list,header):
                excludeded_keys.append(header)
                pass
            else :
                if re.search(excl_add,header):
                    
                    spisok = header.split(' ')
                    spisok.remove(excl_add)
                # (spisok.index(keysss))
                    header=' '.join(spisok)
                else:
                    pass

                REALY_NAMES[header] = sheetname
                a = check_threee_moth_ended(pd.read_excel(period,sheet_name=sheetname))
                
                data_dict.setdefault(header, []).append(a)

# print('OUT' ,len(excludeded_reports))
# for b,i in enumerate(excludeded_reports):
#     print(b+1,i)
# print('______________')

Wall time: 48.9 s


# Look at all selected Reports

In [14]:
print('OUTPUT',len(data_dict),'|||', 'INPUT',len(wb.sheetnames))   
for b,i in enumerate(REALY_NAMES.keys()):
    print(b+1,i)

OUTPUT 152 ||| INPUT 62
1 Document and Entity Information
2 CONDENSED CONSOLIDATED RESULTS OF OPERATIONS - USD ($) shares in Thousands, $ in Millions
3 CONDENSED CONSOLIDATED COMPREHENSIVE INCOME (LOSS) - USD ($) $ in Millions
4 CONDENSED CONSOLIDATED BALANCE SHEETS - USD ($) $ in Millions
5 CONDENSED CONSOLIDATED BALANCE SHEETS (Parenthetical) - USD ($) $ in Millions
6 CONDENSED CONSOLIDATED CASH FLOWS - USD ($) $ in Millions
7 Pension Benefit Plans (Tables)
8 Segment Information (Details) - segment
9 Segment Information - Reportable Segments (Details) - USD ($) $ in Millions
10 Segment Information - Total Assets (Details) - USD ($) $ in Millions
11 Revenue (Details) - USD ($) $ in Millions
12 Credit Losses (Details) - USD ($) $ in Millions
13 Inventories (Details) - USD ($) $ in Millions
14 Derivative Instruments - Derivatives and Hedges (Details) - Cash Flow Hedges - USD ($) $ in Millions
15 Derivative Instruments - Balance Sheet Classification (Details) € in Millions, $ in Millions

# Select Report type

In [46]:
# Just to see Reports where have 10 or more
[f'{frame_name} {len(data_dict[frame_name])}' for frame_name in REALY_NAMES.keys() if len(data_dict[frame_name])>8]

['CONDENSED CONSOLIDATED BALANCE SHEETS - USD ($) $ in Millions 12',
 'CONDENSED CONSOLIDATED BALANCE SHEETS (Parenthetical) - USD ($) $ in Millions 12',
 'CONDENSED CONSOLIDATED CASH FLOWS - USD ($) $ in Millions 12',
 'Pension Benefit Plans (Tables) 11',
 'Segment Information - Total Assets (Details) - USD ($) $ in Millions 12',
 'Revenue (Details) - USD ($) $ in Millions 15',
 'Inventories (Details) - USD ($) $ in Millions 16',
 'Derivative Instruments - Balance Sheet Classification (Details) € in Millions, $ in Millions 14',
 'Derivative Instruments - Effects of Derivative Instruments (Details) - USD ($) $ in Millions 12',
 'Debt (Details) $ / shares in Units, € in Millions, $ in Millions 16',
 'Contingencies - Asbestos (Details) 16',
 'Accumulated Other Comprehensive Loss (Details) - USD ($) $ in Millions 11',
 'Earnings Per Share (Details) - USD ($) $ / shares in Units, $ in Millions 16',
 'Supplemental Cash Flow Information (Details) - USD ($) $ in Millions 16',
 'CONSOLIDATED R

In [53]:
# finder = Report name
finder = "Inventories (Details) - USD ($) $ in Millions"
print(f'App have found {len(data_dict[finder])} documents from\n{REALY_NAMES[finder]}')

App have found 16 documents from
Inventories (Details)


# Prepare_Data

In [29]:
from collections import Counter
def convert_column_index_to_unique(frame):
    war_dict = {}
    
    mylist = frame[frame.columns[0]].to_list()


    [war_dict.update({k:0}) for k,v in Counter(mylist).items()]

    pure_list = []
    for name in mylist:
#         name = re.sub('\d','',name)
        if name not in war_dict.keys():
            
            pure_list.append(name)
        else:
            
            war_dict[name] = war_dict[name]+1
            
            new_name = f'{name}{str(war_dict[name])}'
            pure_list.append(new_name.replace('  ',' '))
    return pure_list

In [30]:
def convert_to_finance_period_x(period_input):
    try:
        perods_dict = {'Nov':'Q_1 - 12','Oct':'Q_1 - 11','Sep':'Q_1 - 10','Aug':'Q_2 - 9','Jul':'Q_2 - 8','Jun':'Q_2 - 7','May':'Q_3 - 6','Apr':'Q_3 - 5','Mar':'Q_3 - 4','Feb':'Q_4 - 3','Dec':'Q_4 - 2','Jan':'Q_4 - 1'}
    
#         'Apr', 'Aug', 
        year = re.findall(r'\d\d\d\d',period_input)[0]
        quartal = perods_dict[re.findall(r'\D\D\D',period_input)[0]]
        
    
        return str(year + ' / ' + quartal)
    except:
        return period_input

In [31]:
def exceptions_rows_delete(frame):
    exceptions_rows = [
    'Common Stock$'
    ]
    list_for_delete = []
    for index,row in enumerate(frame.iloc[:,0].to_list()):
    
        for exception_r in exceptions_rows:
    
            if re.match(exception_r,row):
                print(index)
                list_for_delete.append(index)
            
        
        
    return list_for_delete

In [32]:
def cocncat_mean_rows(frame):
#     new_f = frame.fillna(0)
    
    for row in frame.iloc[:,0]:
        if int(re.search('\d',row)[0])>1:
            pass
        
#             print('here')
#     if sum([pd.Series(frame[col]) for col in new_f.columns[1:]])
    
    
    return frame

In [33]:
def pop_unneed_numbers(frame):
    
   
    withoud_number_list = [re.sub('\d','',row) for row in frame.iloc[:,0]]
    return withoud_number_list

In [34]:
def scrub_one_period(frame):
    frame.iloc[:,0] = pop_unneed_numbers(frame)
    frame.iloc[:,0] = convert_column_index_to_unique(frame) # assigned unque Value for 0 Column
    frame.columns = list(map(lambda date: convert_to_finance_period_x(date),frame.columns)) # convert for normal periods
    
#     w = exceptions_rows_delete(frame)
    return frame

In [35]:
def clear_set(ar):
    ar[ar.columns[0]] = ar[ar.columns[0]].apply(lambda x: x[0:75])
    
    return ar

In [36]:
def clear_frame(frame):
    if len(str(frame.columns[0]))<4 :
        print('here')   
        frame.index=frame.iloc[:,0]
        frame.index.name = 'Category'
        frame.drop(frame.columns[[0]],axis='columns',inplace=True)
    else:
        pass
    return frame
    

In [37]:
def concat_tables(key_name):
    frame_size = len(data_dict[key_name])-1
    structure = list(map(lambda dirty_drame:scrub_one_period(dirty_drame),data_dict[key_name]))
    def recurs(x):
       
        if x ==0:
            col_name = structure[x].columns[0]
            return (structure[x]).merge((structure[x+1]),on=key_name,how='outer',suffixes=('', '_y'))
        
            
        else:
            col_name = structure[x].columns[0]
            
            return recurs(x-1).merge((structure[x-1]),on=key_name,how='outer',suffixes=('', '_y'))
        
    frame = recurs(frame_size)
    
    try:
        
        if frame_size==1:
            return (structure[0])
        else:
            frame = recurs(frame_size)
            frame.drop(frame.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
        
            return frame
    except Exception as ex:
        return ex
    


# GO

In [56]:
conc = concat_tables(finder)
new = conc[conc.columns[1:].sort_values(ascending=True)]
new.insert(0,conc.columns[0],conc[conc.columns[0]])

In [57]:
new

,Inventories (Details) - USD ($) $ in Millions,2016 / Q_4 - 2,2017 / Q_1 - 10,2017 / Q_2 - 7,2017 / Q_3 - 4,2017 / Q_4 - 2,2018 / Q_1 - 10,2018 / Q_2 - 7,2018 / Q_3 - 4,2018 / Q_4 - 2,2019 / Q_1 - 10,2019 / Q_2 - 7,2019 / Q_3 - 4,2019 / Q_4 - 2,2020 / Q_1 - 10,2020 / Q_2 - 7,2020 / Q_3 - 4,2020 / Q_4 - 2,2021 / Q_2 - 7,2021 / Q_3 - 4
0,Inventories1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Finished goods1,827.0,875.0,879.0,889.0,873.0,808.0,821.0,901.0,849.0,836.0,838.0,868.0,872.0,625.0,833.0,884.0,675.0,636.0,672.0
2,Raw materials1,118.0,129.0,130.0,124.0,122.0,119.0,123.0,120.0,125.0,134.0,131.0,126.0,128.0,121.0,129.0,121.0,129.0,122.0,119.0
3,Operating supplies1,38.0,42.0,40.0,38.0,41.0,46.0,41.0,44.0,44.0,45.0,46.0,44.0,45.0,36.0,42.0,42.0,37.0,38.0,36.0
4,Inventories2,983.0,1046.0,1049.0,1051.0,1036.0,973.0,985.0,1065.0,1018.0,1015.0,1015.0,1038.0,1045.0,782.0,1004.0,1047.0,841.0,796.0,827.0
